In [1]:
from langchain_huggingface import HuggingFacePipeline
from langchain.memory import ConversationBufferMemory
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
from langchain_core.prompts import PromptTemplate
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.output_parsers import StrOutputParser
from huggingface_hub import login
import os
import dotenv 

dotenv.load_dotenv()
login(token=os.environ.get("HUGGINGFACE_TOKEN"))

model_id = "allganize/Llama-3-Alpha-Ko-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(

    model_id,
    torch_dtype="auto",
    device_map="auto",
)
pipe = pipeline(
    "text-generation", 
    model=model, 
    tokenizer=tokenizer, 
    max_new_tokens=1024,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id,
    do_sample=True,
    temperature=0.1,
    top_p=0.9,
)
llm = HuggingFacePipeline(pipeline=pipe)

# prompt = PromptTemplate.from_template("""<|start_header_id|>system<|end_header_id|>
#                                       너는 user 질문에 정보의 내용대로 답하는 assistant야. user가 질문하는 내용을 정보를 이용해서 정확하고 최대한 자세하게 답해. 질문 내용이 정보에 없으면 '이 질문은 답변할 수 없습니다'라고 답해. 한국어(Korean)로 대답해.<|eot_id|><|start_header_id|>user<|end_header_id|>
#                                       안녕하세요!<|eot_id|><|start_header_id|>assistant<|end_header_id|>
#                                       안녕! 만나서 반가워요!<|eot_id|><|start_header_id|>user<|end_header_id|>
#                                       ### 질문: {question}
#                                       ### 답변<|eot_id|><|start_header_id|>assistant<|end_header_id|>
#                                       """)

prompt = PromptTemplate.from_template("""<|start_header_id|>system<|end_header_id|>
                                      너는 user 질문에 정보의 내용대로 답하는 assistant야. user가 질문하는 내용을 정보를 이용해서 정확하고 최대한 자세하게 답해. 질문 내용이 정보에 없으면 '이 질문은 답변할 수 없습니다'라고 답해. 한국어(Korean)로 대답해.
                                      ### 질문: {question}
                                      ### 답변<|eot_id|>
                                      <|start_header_id|>assistant<|end_header_id|>
                                      """)

chain  = prompt | llm | StrOutputParser()

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /home/aifactory_ds2/.cache/huggingface/token
Login successful


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [2]:
# 세션 기록을 저장할 딕셔너리
store = {}


# 세션 ID를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids : str):
    print(f"[대화 세션ID]: {session_ids}")
    if session_ids not in store:  # 세션 ID가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환


chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,  # 세션 기록을 가져오는 함수
    input_messages_key="question",  # 사용자의 질문이 템플릿 변수에 들어갈 key
    history_messages_key="history",  # 기록 메시지의 키
    # output_messages_key="output_message"
)
output = chain_with_history.invoke(
    # 질문 입력
    {"question": "내 이름은 테디노트라구해"},
    # 세션 ID 기준으로 대화를 기록합니다.
    config={"configurable": {"session_id": "history"}},
)
print(output.split("### 답변<|eot_id|><|start_header_id|>assistant<|end_header_id|>")[1].replace("\n                                       ",""))
output = chain_with_history.invoke(
    # 질문 입력
    {"question": "내 이름이 뭐라고?"},
    # 세션 ID 기준으로 대화를 기록합니다.
    config={"configurable": {"session_id": "history"}},
)
print(output.split("### 답변<|eot_id|><|start_header_id|>assistant<|end_header_id|>")[1].replace("\n                                       ",""))

[대화 세션ID]: history
테디노트라는 이름은 특별하고 멋진 이름이에요. 테디노트라는 이름을 가진 분은 누구나 특별하고 독특한 존재임을 증명해줘요. 이름에 담긴 의미나 특별한 기억이 있으신가요?assistant
이 질문은 답변할 수 없습니다assistant
이 질문은 답변할 수 없습니다assistant
이 질문은 답변할 수 없습니다assistant
이 질문은 답변할 수 없습니다assistant
이 질문은 답변할 수 없습니다assistant
이 질문은 답변할 수 없습니다assistant
이 질문은 답변할 수 없습니다assistant
이 질문은 답변할 수 없습니다
[대화 세션ID]: history
이 질문은 답변할 수 없습니다
